In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from numpy import mean
import pandas as pd
from numpy import log, dot, e

In [2]:
#!pip install matplotlib
#!pip install sklearn

## Titantic dataset

In [3]:
Xx = pd.read_csv('titanic.csv',nrows=200)
X = Xx[['Pclass','Sex','Siblings/Spouses Aboard','Parents/Children Aboard','Survived']]
X = pd.get_dummies(X)
X['target'] = X['Survived']
X = X.drop(['Survived'],axis=1)

In [4]:
#dataset columns
header = X.columns

## for user generated dataset

In [5]:
def create_random_data(hm_features,obs):

        X = np.ones((obs))

        for i in range(hm_features):
            xs = np.random.rand(obs)*100
            X = np.append(X,xs)

        X = np.reshape(X, (-1,obs)).transpose()
        ys = np.random.randint(0,2,size=obs) # 2 high as it's exclusive

        return X,ys    

In [6]:
s,t = create_random_data(4,10)

## Useful functions

In [7]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset.
    Note - this function expect last column as target variable
    Output - {0:12,1:43} 12 0's and 43 1's
    """
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [8]:
def is_numeric_(val):
    '''
    check if given value is a integer or float
    '''
    return isinstance(val,int) or isinstance(val,float)

In [9]:
#Demo
is_numeric_(2)
#True

is_numeric_("red")
#False

False

## 1st step - Create a question to ask based on feature and threshold value

In [10]:
class question():
    '''
    This is the first step in building the decision tree
    Build a question i.e. Given a column and value, what would be
    the question?
    
    question(1,3)
    >>is Siblings/Spouses Aboard >= 3 
    
    question(2,0)
    >>is Parents/Children Aboard >= 0 
    
    '''
    def __init__(self,col,val):
        self.col = col
        self.val = val
    
    def __repr__(self):
        if is_numeric_(self.val):
            return  "is %s >= %s " % (header[self.col],self.val)
        else: return "is %s == %s " % (header[self.col],self.val)
    
    def match(self,df):
        '''
        compare feature value in df to threshold value we have created 
        '''
        v = df[self.col]
        if is_numeric_(v):
            return v >= self.val
        else:
            return v == self.val

## Part 2 - Partition dataset based on question & output 2 nodes

In [11]:
def partition_my_data(df,ask_question):
    '''
    2nd part is partition dataset based on question
    
    partition_my_data(df,question(column = 2,value = 0))
    >> create two df i.e. true rows where question conditon match
    and false rows where question doesn't match
    
    len(tr), len(fr)
    >>(148,52)
    
    '''
    true_rows, false_rows = [] , []
    for r in df:
        if ask_question.match(r):
            true_rows.append(r)
        else:
            false_rows.append(r)
            
    return true_rows,false_rows


## 3rd part - Calculate Information gain for all features & threshold

In [12]:
def cal_gini(df):
    '''
    Gini formula = 1 - (prob_of_1)^2 - (prob_of_0)^2
    
    Example
    cal_gini(X) #parent node gini
    >> 0.545
    
    '''
    impurity = 1
    counts = class_counts(df) # calculate y distribution of df
    
    for c in counts:
        prob_of_c = counts[c] / float(len(df))
        impurity -= (prob_of_c)**2
        
    return impurity

In [13]:
def info_gain(left_rows,right_rows,current_info):
    '''
    Information gain = current_information_gain [parent node] - left node - right node
    Information gain = current_info - p(gini(left)) - (1-p)gini(right)
    where p is no of rows in left/ total rows (left+right)
    
    Example
    info_gain(false_rows, true_rows, current_gini)
    >> 0.43
    '''
    p = float( len(left_rows)/ (len(left_rows)+len(right_rows)))
    
    return current_info - (p * cal_gini(left_rows)) - ((1-p) * cal_gini(right_rows))


## 4th part - Find best feature and threshold using information gain
### Note - 4th part reach out to part 1-3 so everything

In [14]:
def best_split(x):
    '''
    finding the best question to ask i.e. best feature and best threshold
    calculate the IV
    
    Example
    
    best_split(X.to_numpy())
    >>(0.15340430723878157, is Sex_female == 0 )
    '''
    #Function output values best gain and best question
    best_gain = 0 #to track the IV values
    best_question = None #best question based on feature and threshold
    
    
    #Parent node gini value
    currrent_uncertainity = cal_gini(x)
    n_feature = len(x[0]) - 1 # -1 for target feature
    
    for col in range(n_feature): # for each feature
        u_val = set([xi[col] for xi in x]) # unique values of features
        
        for u_v in u_val:
            
            #1st part
            my_question = question(col,u_v) #create condition
            
            #2nd part
            true_rows, false_rows = partition_my_data(x,my_question) #split data based on question
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            
            #3rd part
            my_gain = info_gain(true_rows,false_rows,currrent_uncertainity) # find IV
            
            #to track best gain value and feature
            if my_gain > best_gain:
                best_gain, best_question = my_gain, my_question
                
    return best_gain,best_question    

## 5th part - Create a decision node and leaf to hold parent and child nodes

In [15]:
class leaf():
    '''
    leaf node hold the original target value distribution
    {0:12,1:50} Means there are 12 0's and 50 1's in training data
    
    >> This will be the leaf node which will hold distribution of target variable based on all
    condition
    '''
    
    def __init__(self,rows):
        self.predictions = class_counts(rows)    

In [16]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    question -> from 1st part
    true branch and false branch --> from 2nd part
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

## 6th part - train decision tree

In [17]:
def build_tree(x):
    '''
    rules of recursion: 1) believe that it works 2) start by checking base case
    3) prepare for giant case
    
    Summary of this function
    1. Find best feature and threshold [question]
    2. Split data on best feat and thres [quest]
    3. recursively do 1 & 2 for split data
    4. Return decision node with best feat and threshold, true branch and false branch
    '''
    
    # find best question and return it's IV
    gain, question = best_split(x) #This is 4th part which internally calls 1-3 part
    
    #base case when no split possible
    # we can't split anymore so time for leaf node and store y distribution
    if gain == 0:
        return leaf(x)
    
    #we have best question so now time to split
    #partition data on best question
    true_rows, false_rows = partition_my_data(x,question) # part - 2
    
    #recursively build tree
    true_branch = build_tree(true_rows)
    
    #recursively build tree
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question,true_branch,false_branch)

## 7th part - Print fitted decision tree

In [18]:
def print_tree(node, sp = ""):
    '''
    print tree
    '''
    
    #if it is a leaf, print y distribution
    if isinstance(node, leaf):
        print(sp,"predict",node.predictions)
        return 
    
    #question at node
    print(sp,str(node.question))
    
    #print true branch recursively
    print(sp,"--> True")
    print_tree(node.true_branch,sp+" ")
    
    #print true branch recursively
    print(sp,"--> False")
    print_tree(node.false_branch,sp+" ")
    

In [19]:
my_tree = build_tree(X.to_numpy())

In [20]:
print_tree(my_tree)

 is Sex_female == 0 
 --> True
  is Pclass == 2 
  --> True
   is Parents/Children Aboard == 0 
   --> True
    is Siblings/Spouses Aboard == 0 
    --> True
     predict {1: 2, 0: 12}
    --> False
     predict {0: 4}
   --> False
    is Siblings/Spouses Aboard == 2 
    --> True
     predict {1: 1}
    --> False
     predict {1: 2, 0: 2}
  --> False
   is Siblings/Spouses Aboard == 0 
   --> True
    is Parents/Children Aboard == 2 
    --> True
     predict {0: 1, 1: 1}
    --> False
     is Pclass == 3 
     --> True
      is Parents/Children Aboard == 0 
      --> True
       predict {0: 47, 1: 6}
      --> False
       predict {0: 2}
     --> False
      is Parents/Children Aboard == 0 
      --> True
       predict {0: 11, 1: 3}
      --> False
       predict {0: 5, 1: 1}
   --> False
    is Siblings/Spouses Aboard == 1 
    --> True
     is Parents/Children Aboard == 1 
     --> True
      predict {1: 1, 0: 2}
     --> False
      is Pclass == 1 
      --> True
       predict {

## 9th part - classify new sample

In [26]:
def classify_new_sample(sample,node):
    '''
    '''
    
    #when we have reached the leaf node
    if isinstance(node,leaf):
        return node.predictions
    
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(sample):
        return classify_new_sample(sample,node.true_branch)
    
    else:
        return classify_new_sample(sample,node.false_branch)

In [27]:
sam = X.to_numpy()[0]

In [30]:
for i in range(0,5):
    print(classify_new_sample(X.to_numpy()[i],my_tree))

{0: 3, 1: 1}
{1: 5}
{1: 13, 0: 3}
{1: 5}
{0: 47, 1: 6}
